In [ ]:
import numpy as np
from google.colab import drive
from tensorflow.keras.utils import to_categorical

drive.mount('/content/drive')
train_data_path = "/content/drive/My Drive/기계학습 실습/train_data.npy" # Your train data file
train_label_path = "/content/drive/My Drive/기계학습 실습/train_label.npy" # Your train label file
test_data_path = "/content/drive/My Drive/기계학습 실습/test_data.npy" # Your train data file
test_label_path = "/content/drive/My Drive/기계학습 실습/test_label2.npy" # Your train label file

x_train = np.load(train_data_path, allow_pickle=True) # (2682,) max = 502
y_train = np.load(train_label_path, allow_pickle=True)  #(2682,)
x_test = np.load(test_data_path, allow_pickle=True) # (2690,) max = 432
y_test = np.load(test_label_path, allow_pickle=True) # (2690,)

# 라벨 정수 인코딩
label = []
train_label = []
test_label = []
rating = ['18k', '17k', '16k', '15k', '14k', '13k', '12k', '11k', '10k', 
          '9k', '8k', '7k', '6k', '5k', '4k', '3k', '2k', '1k',
          '1d', '2d', '3d', '4d', '5d', '6d', '7d', '8d', '9d']
for i in range(2682):
    label.append(rating.index(y_train[i])) 

for i in range(2690):
    label.append(rating.index(y_test[i]))

label = np.array(label) 
label= to_categorical(label)

length = 50
train_encoded = np.zeros((5372,length,19,19))

for i in range(2682):
    for j in range(length):
        try:
            for row in range(0,19):
                for col in range(0,19):
                    if x_train[i][j][0][row][col] == -1 :
                        train_encoded[i][j][row][col] = 200
                    elif x_train[i][j][0][row][col] == 1 :
                        train_encoded[i][j][row][col] = 100
                    else:
                        train_encoded[i][j][row][col] = x_train[i][j][0][row][col]
        except:
            continue

for i in range(2690):
    for j in range(length):
        try:
            for row in range(0,19):
                for col in range(0,19):
                    if x_test[i][j][0][row][col] == -1 :
                        train_encoded[i+2682][j][row][col] = 200
                    elif x_test[i][j][0][row][col] == 1 :
                        train_encoded[i+2682][j][row][col] = 100
                    else:
                        train_encoded[i+2682][j][row][col] = x_test[i][j][0][row][col]
        except:
            continue

query_path = "/content/drive/My Drive/기계학습 실습/Test2_data.npy" 
query = np.load(query_path, allow_pickle=True) 

query_list = np.zeros((2690,length,19,19))

for i in range(2690):
    for j in range(length):
        try:
            for row in range(0,19):
                for col in range(0,19):
                    if query[i][j][0][row][col] == -1 :
                        query_list[i][j][row][col] = 200
                    elif query[i][j][0][row][col] == 1 :
                        query_list[i][j][row][col] = 100
                    else:
                        query_list[i][j][row][col] = query[i][j][0][row][col]
        except:
            continue
            
query_list = query_list.reshape((2690,length,19,19,1))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
from google.colab import drive
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, AveragePooling2D, ConvLSTM2D, Flatten, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

train_encoded = train_encoded.reshape(5372,length,19,19,1)
x_train, x_test, y_train, y_test = train_test_split(train_encoded, label, test_size=.1, random_state=777)

with tf.device('/device:GPU:0'):
    
    predict_list = []
    for i in range(10):
        model = Sequential()
        
        model.add(ConvLSTM2D(filters = 64, kernel_size = (4,4), return_sequences = True, input_shape = (length, 19, 19, 1)))  
        model.add(Activation('relu'))    
        model.add(TimeDistributed(AveragePooling2D(2,2)))
        
        model.add(ConvLSTM2D(filters = 64, kernel_size = (4, 4), return_sequences = True))   
        model.add(Activation('relu'))
        model.add(TimeDistributed(AveragePooling2D(5,5)))   

        model.add(Flatten())
        
        model.add(Dense(960))
        model.add(Activation('relu'))

        model.add(Dense(480))
        model.add(Activation('relu'))

        model.add(Dense(240))
        model.add(Activation('relu'))
        
        model.add(Dense(27, activation = "softmax"))
        
        model.summary()
        
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=12)
        mc = ModelCheckpoint('ML_team1.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
        model.compile(loss='categorical_crossentropy', optimizer= 'RMSProp' , metrics=['acc'])

        history = model.fit(x_train, y_train, batch_size=16, callbacks=[es, mc], epochs=90, validation_data=(x_test, y_test))

        predict_list.append(model.predict(query_list))    
    

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 50, 16, 16, 64)    266496    
_________________________________________________________________
activation (Activation)      (None, 50, 16, 16, 64)    0         
_________________________________________________________________
time_distributed (TimeDistri (None, 50, 8, 8, 64)      0         
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 50, 5, 5, 64)      524544    
_________________________________________________________________
activation_1 (Activation)    (None, 50, 5, 5, 64)      0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 1, 1, 64)      0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0

In [ ]:
predict_result = (predict_list[0] + predict_list[1] + predict_list[2] +predict_list[3] +predict_list[4] +predict_list[5] +predict_list[6] +predict_list[7] +predict_list[8] + predict_list[9])/10
predict_label = []
for k in predict_result:        
    predict_label.append(k.argmax())

pred_test_label_txt = list_data = [str(rating[int(a)]).strip('\n\r') for a in predict_label]
print(pred_test_label_txt)


['7d', '7k', '14k', '14k', '12k', '7d', '10k', '17k', '7d', '4k', '2d', '14k', '9k', '17k', '9k', '9d', '9k', '6k', '2d', '4d', '16k', '7d', '5k', '16k', '9k', '9d', '1k', '17k', '9k', '16k', '5d', '4k', '17k', '8d', '3d', '9d', '8d', '9d', '15k', '18k', '1d', '1d', '4d', '10k', '15k', '4k', '3k', '3d', '14k', '5d', '8k', '6k', '4k', '12k', '3k', '14k', '10k', '2d', '4k', '5d', '5k', '1d', '1k', '9k', '8d', '5d', '10k', '9k', '1d', '8d', '5k', '17k', '6d', '7d', '2d', '1d', '4d', '15k', '5d', '9d', '1d', '14k', '3k', '9d', '5d', '9d', '1d', '5d', '9k', '9d', '9k', '12k', '6k', '4k', '3k', '9d', '7d', '8d', '5k', '16k', '14k', '15k', '10k', '5k', '5k', '15k', '7d', '9d', '6k', '1d', '12k', '8d', '3k', '3k', '6d', '17k', '9d', '5d', '6k', '7d', '17k', '8d', '5k', '17k', '10k', '15k', '5d', '8d', '3k', '12k', '2k', '9d', '16k', '8d', '12k', '1d', '15k', '8d', '9k', '9d', '8d', '6k', '6k', '9d', '17k', '1d', '7d', '7d', '9k', '9k', '8d', '1k', '10k', '3k', '11k', '5d', '7d', '11k', '6k', '

In [ ]:
print(predict_result)
np.save("/content/drive/My Drive/기계학습 실습/prob.npy",predict_result)

[[3.8158994e-03 5.6418674e-03 6.1337375e-03 ... 7.7847287e-02
  6.7103781e-02 4.7074198e-03]
 [2.3481127e-02 2.0305295e-02 2.0243328e-02 ... 4.5356061e-02
  4.8758093e-02 6.7212708e-02]
 [2.5557628e-02 5.1133417e-02 7.4759021e-02 ... 1.9242188e-04
  2.1667576e-04 4.9033833e-06]
 ...
 [7.2468533e-03 2.4725949e-02 2.9566064e-02 ... 1.7057458e-02
  3.1336464e-02 1.8883873e-05]
 [7.8889832e-02 1.8980888e-01 2.1987936e-01 ... 4.5925967e-04
  6.1732024e-04 9.8199530e-07]
 [3.5630316e-03 5.9804763e-03 1.4698682e-02 ... 1.0391606e-02
  9.1353329e-03 4.9150252e-04]]


In [ ]:
!pip install pycryptodomex --no-binary :all:
import json
from base64 import b64encode
from Cryptodome.Cipher import AES
from Cryptodome.Util.Padding import pad

def read_txt(fileName):
    with open(fileName, 'rt') as f:
        list_data = [a.strip('\n\r') for a in f.readlines()]
    return list_data

def write_json(fileName, data):
    with open(fileName, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def load_key(key_path):
    with open(key_path, "rb") as f:
        key = f.read()
    return key

def encrypt_data(key_path, ans_list, encrypt_store_path='ans.json'):
    key = load_key(key_path)
    data = " ".join([str(i) for i in ans_list])
    encode_data = data.encode()
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(encode_data, AES.block_size))
    iv = b64encode(cipher.iv).decode('utf-8')
    ct = b64encode(ct_bytes).decode('utf-8')
    write_json(encrypt_store_path, {'iv':iv, 'ciphertext':ct})

if __name__=="__main__":
    # 1.이메일을 통해서 전달 받은 키 파일의 경로 입력
    key_path = "/content/drive/My Drive/기계학습 실습/team1.pem"
    # 2. 예측한 결과를 텍스트 파일로 저장했을 경우 리스트로 다시 불러오기
    # 본인이 원하는 방식으로 리스트 형태로 예측 값을 불러오기만 하면 됨(순서를 지킬것)
    #raw_ans_path = "ans.txt"
    #ans = read_txt(raw_ans_path)
    ans = pred_test_label_txt
    # 3. 암호화된 파일을 저장할 위치
    encrypt_ans_path = "/content/drive/My Drive/기계학습 실습/ans.json"
    # 4. 암호화!(pycrytodome 설치)
    encrypt_data(key_path, ans, encrypt_ans_path)

     |████████████████████████████████| 3.8MB 14.2MB/s 
Skipping wheel build for pycryptodomex, due to binaries being disabled for it.
    Running setup.py install for pycryptodomex ... done
